<a href="https://colab.research.google.com/github/arijitmazumdar/colab-repo/blob/main/duckdb_arrow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task
Create a sample Python program that loads a CSV file using DuckDB, converts the data to Apache Arrow format, and stores it in a SQLite database.

## Setup

### Subtask:
Install necessary libraries (duckdb, pyarrow, pandas, sqlite3).


**Reasoning**:
Install the necessary libraries using pip. SQLite3 is usually included in Python's standard library, so we don't need to install it separately via pip.



In [7]:
%pip install duckdb pyarrow pandas

## Create sample csv

### Subtask:
Create a dummy CSV file for testing.


**Reasoning**:
Create a pandas DataFrame with sample data and save it to a CSV file as instructed.



In [8]:
import pandas as pd
import numpy as np

# Create a pandas DataFrame
data = {
    'integer_col': [1, 2, 3, 4, 5],
    'string_col': ['apple', 'banana', 'cherry', 'date', 'elderberry'],
    'float_col': [1.1, 2.2, 3.3, 4.4, 5.5],
    'date_col': pd.to_datetime(['2023-01-01', '2023-01-02', '2023-01-03', '2023-01-04', '2023-01-05'])
}
df = pd.DataFrame(data)

# Save the DataFrame to a CSV file without the index
df.to_csv('sample_data.csv', index=False)

print("Dummy CSV file 'sample_data.csv' created successfully.")

Dummy CSV file 'sample_data.csv' created successfully.


## Load csv with duckdb

### Subtask:
Use DuckDB to load the data from the created CSV file.


**Reasoning**:
The subtask requires loading the data from the created CSV file using DuckDB. This involves importing the `duckdb` library, connecting to a DuckDB database, and executing a SQL query to read the CSV.



In [9]:
import duckdb

# Connect to an in-memory DuckDB database
con = duckdb.connect(database=':memory:', read_only=False)

# Execute a SQL query to read the 'sample_data.csv' file
duckdb_result = con.execute("SELECT * FROM 'sample_data.csv'").fetchall()

# Print the result to verify
print(duckdb_result)

# Close the connection (optional for in-memory, but good practice)
con.close()

[(1, 'apple', 1.1, datetime.date(2023, 1, 1)), (2, 'banana', 2.2, datetime.date(2023, 1, 2)), (3, 'cherry', 3.3, datetime.date(2023, 1, 3)), (4, 'date', 4.4, datetime.date(2023, 1, 4)), (5, 'elderberry', 5.5, datetime.date(2023, 1, 5))]


## Convert to arrow

### Subtask:
Convert the data loaded by DuckDB into Apache Arrow format.


**Reasoning**:
Connect to DuckDB, execute a query to read the CSV, fetch the result as an Arrow object, store it, and close the connection as per the instructions.



In [10]:
import duckdb
import pyarrow as pa

# Connect to an in-memory DuckDB database
con = duckdb.connect(database=':memory:', read_only=False)

# Execute a SQL query to read the 'sample_data.csv' file and fetch as Arrow
arrow_table = con.execute("SELECT * FROM 'sample_data.csv'").arrow()

# Store the resulting Arrow object in a variable (already done in the previous step)
# Display the Arrow table to verify
print("Data loaded into Apache Arrow format:")
print(arrow_table)

# Close the DuckDB connection
con.close()

Data loaded into Apache Arrow format:
pyarrow.Table
integer_col: int64
string_col: string
float_col: double
date_col: date32[day]
----
integer_col: [[1,2,3,4,5]]
string_col: [["apple","banana","cherry","date","elderberry"]]
float_col: [[1.1,2.2,3.3,4.4,5.5]]
date_col: [[2023-01-01,2023-01-02,2023-01-03,2023-01-04,2023-01-05]]


## Store in sqlite

### Subtask:
Store the data from the Arrow format into a SQLite database.


**Reasoning**:
Store the data from the Arrow format into a SQLite database following the instructions.



In [12]:
import sqlite3
import pandas as pd

# Connect to a SQLite database file
conn = sqlite3.connect('sample_data.db')

# Convert the Arrow Table to a pandas DataFrame
df = arrow_table.to_pandas()

# Use the .to_sql() method to write the data to a table in the SQLite database
df.to_sql('sample_table', conn, if_exists='replace', index=False)

# Commit the changes to the database
conn.commit()

# Close the SQLite database connection
conn.close()

print("Data successfully stored in 'sample_data.db' in table 'sample_table'.")

Data successfully stored in 'sample_data.db' in table 'sample_table'.


## Verify data in sqlite

### Subtask:
Read the data from the SQLite database to verify it was stored correctly.


**Reasoning**:
Read the data from the SQLite database to verify it was stored correctly by connecting to the database, selecting all data from the table, and printing the results.



In [13]:
import sqlite3

# Connect to the SQLite database file
conn = sqlite3.connect('sample_data.db')

# Create a cursor object
cursor = conn.cursor()

# Execute a SQL query to select all data from the 'sample_table'
cursor.execute("SELECT * FROM sample_table")

# Fetch all the results
results = cursor.fetchall()

# Print the fetched data
print("Data read from SQLite database:")
for row in results:
    print(row)

# Close the cursor and the database connection
cursor.close()
conn.close()

Data read from SQLite database:
(1, 'apple', 1.1, '2023-01-01')
(2, 'banana', 2.2, '2023-01-02')
(3, 'cherry', 3.3, '2023-01-03')
(4, 'date', 4.4, '2023-01-04')
(5, 'elderberry', 5.5, '2023-01-05')


## Summary:

### Data Analysis Key Findings

*   The necessary libraries (duckdb, pyarrow, pandas, and the built-in sqlite3) were available in the environment, allowing the program to proceed.
*   A sample CSV file named `sample_data.csv` was successfully created with columns of different data types (integer, string, float, date).
*   DuckDB was successfully used to load the data from `sample_data.csv`, and the results were retrieved.
*   The data loaded by DuckDB was successfully converted into an Apache Arrow `Table` using the `.arrow()` method.
*   The Apache Arrow `Table` was converted to a pandas DataFrame and then successfully stored in a SQLite database file named `sample_data.db` within a table called `sample_table`.
*   Verification confirmed that the data was correctly written to the SQLite database by reading it back and printing it.

### Insights or Next Steps

*   This process demonstrates a flexible way to move data between different storage formats and tools (CSV -> DuckDB -> Arrow -> SQLite), which can be useful in various data pipelines.
*   The use of Apache Arrow as an intermediate format can be beneficial for performance when dealing with larger datasets or integrating with other Arrow-compatible systems.
